# *[Formação Cientista de Dados - Data Science Academy][1]*
## Projeto 1 - Detecção de Fraudes no Tráfego de Cliques em Propagandas de Aplicações Mobile  
[1]:https://www.datascienceacademy.com.br/bundle/formacao-cientista-de-dados

___
### > **Etapa 1**: Definindo o Problema

> [TalkingData][1] é a maior plataforma de Big Data independente da China. Lidam com mais 3 bilhões de cliques diários, 90% dos quais são potencialmente fraudulentos.  
  
> O problema de negócio é prever se um usuário fará o download de um aplicativo após o clique em um anúncio para dispositivos móveis.  
  
> **_Objetivo:_** Contruir um modelo para prever se um clique é ou não fraudulento

[1]:https://www.talkingdata.com/

---
### > **Etapa 2**: Obtendo Dados

> Os dados estão disponíveis por meio da comptição no kaggle: [TalkingData AdTracking Fraud Detection Challenge][1]

[1]:https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/data

> Arquivos disponíveis:  
  
* **train.csv** - Dados para treino.
* **train_sample.csv** - Amostra selecionada aleatóriamente de 100,000 linhas dos dados de treino.
* **test.csv** - Dados de teste.
* **sampleSubmission.csv** - Amostra de dados para subimssão na competição.
* **test_supplement.csv** - Dados de teste disponibilizados por acidente. Os dados de teste são um subset desde arquivo.

---
### > **Etapa 3**: Análise Exploratória

> Descrição dos Atributos
1. **_Dados de Treino_**
* **ip**: endereço de ip do click.
* **app**: id do aplicativo anunciado.
* **device**: id do tipo de dispositivo móvel do usuário (e.g., iphone 6 plus, iphone 7, huawei mate 7, etc.).
* **os**: id da versão do OS do usuário móvel.
* **channel**: id do canal do editor do anúncio.
* **click_time**: data/hora do click (UTC).
* **attributed_time**: horário do download do aplicativo, após clique no anúncio, caso seja realizado.
* **is_attributed**: atributo alvo para previsão, indicando se o aplicativo foi ou não baixado

2. **_Dados de Teste:_** similar ao de treino, com as seguintes diferenças
* **click_id**: referência para previsão
* **is_attributed**: não incluído

In [1]:
# Inicializando o pacotes 
library(readr)           # [readr] para leitura de grandes arquivos de dados
library(tidyverse)       # Metapacote para todos os incluidos no tidyverse
library(Amelia)          # Pacote para análise de dados faltantes
library(corrplot)        # Pacote para análise de correlação
library(caret)           # Pacote para preprocessamento dos dados
library(ROSE)            # Pacote para balanceamento das classes
library(e1071)           # Pacote para aplicação do Naive Bayes
library(ROCR)            # Pacote para avaliação da curva ROC

In [2]:
# Importando dados de treino
train_sample <- read_csv('../input/talkingdata-adtracking-fraud-detection/train_sample.csv', col_names = TRUE, show_col_types = FALSE)

In [3]:
# Analistando as primeiras linhas do dataset
head(train_sample)
tail(train_sample)

In [4]:
### Analisando informações do dataset

# Tipos de Dados
str(train_sample)

In [5]:
# Contagem de valores únicos por coluna 

for (n in colnames(train_sample)){
    print(paste(n, ': ', n_distinct(train_sample[n])))
}

In [6]:
# Os dados correspondem aos cliques realizados entre os dias 06/11/2017 e 09/11/2017
n_distinct(as.Date(train_sample$click_time))
unique(as.Date(train_sample$click_time))

In [7]:
# Contando dados NA por coluna
sapply(train_sample, function(x) sum(is.na(x)/length(x))*100)                     #99,77% dos dados da coluna [attributed_time] é NA
                                                                                  #Somente haverá download quando houver [attributed_time]
       
nrow(train_sample[train_sample$is_attributed == 0, ])/nrow(train_sample) * 100    #Conforme pode ser calculando diretamente
                                                                                  #Tal fato aponta para um forte desbalanceamento na classe alvo

In [19]:
# Verificando o desbalanceamento na variável dependente por meio de um plot de barras
plot(as.factor(train_sample$is_attributed))

In [ ]:
# Visualizando dados faltantes
## Verificamos que somente temos NA na coluna [attributed_time]
missmap(train_sample)

In [ ]:
# Fazendo o boxplot das variaveis independentes X variável v

par(mfrow = c(1,2))
boxplot(data = train_sample,   ip ~ is_attributed)      # Indício da presença de outliers quando is_attributed = 0
boxplot(data = train_sample,   app ~ is_attributed)     # Indício da presença de outliers em ambos os casos
boxplot(data = train_sample,   device ~ is_attributed)  # Indício da presença de outliers em ambos os casos
boxplot(data = train_sample,   os ~ is_attributed)      # Indício da presença de outliers em ambos os casos

In [ ]:
par(mfrow = c(1,2))
boxplot(data = train_sample,   channel ~ is_attributed)           # Sem indício de outliers
boxplot(data = train_sample,   click_time ~ is_attributed)        # Sem indício de outliers e equilibrio entre 0 e 1
boxplot(data = train_sample,   attributed_time ~ is_attributed)   # Como esperado, somente apresenta valores quando a varíavel dependente é igual a 1

In [ ]:
# Plotando o relacionamento entre o momento do Clique e o momento do download
plot(train_sample$attributed_time, train_sample$click_time)

In [ ]:
### Analisando a Correlação entre as variáveis

# Obtendo apenas as colunas numéricas
colunas_numericas <- sapply(train_sample, is.numeric)
colunas_numericas

# Filtrando as colunas numéricas para correlação
data_cor <- cor(train_sample[,colunas_numericas])

head(data_cor)

In [ ]:
corrplot(data_cor, method = 'color')

> Como era de se esperar, existe forte correlação entre as variáveis [*device*], [*os*] e [*app*]

> Correlação positiva fraca entre a variável dependente [*is_attributed*] e as variáveis independentes [*ip*] e [*app*].  
  
> Correlação negativa fraca entre a variável dependente [*is_attributed*] e a variável independente [*channel*].  
  
> Pratiamente ausente a correlação com as variáveis [*device*] e [*os*].


> **Com base na análise dos dados temos que:**
1. As variáveis independentes [*ip*], [*app*], [*device*], [*os*] e [*channel*] são categóricas nominais:
* *Representam uma categoria fixa (e.g. aplicativo, tipo de aparelho ou sistema operacional) cuja a ordem não é relevante*
2. [is_attributed] é categórica nominal com duas possibilidade: 
* *0 caso não tenha sido feito o download e 1 caso tenha sido feito o download*

---
### > **Etapa 4**: Preparação dos Dados


In [ ]:
## Convertendo as variáveis para o tipo categórico
#train_sample[colunas_numericas] <- as.factor(train_sample[colunas_numericas])

train_sample$ip <- as.factor(train_sample$ip)
train_sample$app <- as.factor(train_sample$app)
train_sample$device <- as.factor(train_sample$device)
train_sample$os <- as.factor(train_sample$os)
train_sample$channel <- as.factor(train_sample$channel)
train_sample$is_attributed <- as.factor(train_sample$is_attributed)

In [ ]:
## Dividindo os dados entre treino e teste
indexes <- sample(1:(nrow(train_sample)), size = 0.7 * (nrow(train_sample)))
treino <- train_sample[indexes,]
teste <- train_sample[-indexes,]

In [ ]:
# Fazendo o balanceamento das classes
treino_over <- ovun.sample(is_attributed ~ ip + app + device + os + channel, method = "both", data = treino)$data

In [ ]:
# Verificando o resultado do balanceamento na variável dependente
plot(treino_over$is_attributed)

In [ ]:
str(treino)

---
### > **Etapa 5**: Treinamento do Modelo

In [ ]:
# Criando modelos preditivos baseados em Naive Bayes
modeloV1 <- naiveBayes(is_attributed ~ ip + app + device + os + channel, 
          data = treino)
modeloV2 <- naiveBayes(is_attributed ~ ip + app + channel, 
          data = treino)

In [ ]:
# Criando modelo com os dados balanceados
modelo_balance <- naiveBayes(is_attributed ~ ip + app + device + os + channel, 
          data = treino_over)

In [ ]:
summary(modeloV1)
summary(modeloV2)
summary(modelo_balance )

In [ ]:
# Criando vetor para segundo teste
p2 <- c('ip','app','channel')

# Realiizando as previsões com os modelos
previ1 <- predict(modeloV1, treino[,1:5])
previ2 <- predict(modeloV2, treino[,p2])
previ_balance <- predict(modelo_balance, treino[,1:5])

In [ ]:
# Criando a Confusion Matrix com as previsões
table(pred = previ1, true = treino$is_attributed)
table(pred = previ2, true = treino$is_attributed)
table(pred = previ_balance, true = treino$is_attributed)

In [ ]:
## O modeloV1 apresentou o melhor resultado

# Verificando a média de acertos entre as previsões e o grupo de treino
mean(previ1 == treino$is_attributed)
mean(previ2 == treino$is_attributed)
mean(previ_balance == treino$is_attributed)

In [ ]:
#Aplicando o modelo em sequência para verificação da variação dos resultados

fraction_correct <- rep(NA,10)
for (i in 1:10){
    indexes <- sample(1:(nrow(train_sample)), size = 0.7 * (nrow(train_sample)))
    treino <- train_sample[indexes,]
    teste <- train_sample[-indexes,]
    modeloV1 <- naiveBayes(is_attributed ~ ip + app + device + os + channel, 
          data = treino)
    previ1 <- predict(modeloV1, treino[,1:5])
    fraction_correct[i] <- mean(previ1 == treino$is_attributed)
}

In [ ]:
summary(fraction_correct)
sd(fraction_correct)

---
### > **Etapa 6**: Avaliando do Modelo

In [ ]:
# Realizando a previsão com base no sample de testes
previsao <- predict(modeloV1, teste[,1:5])

In [ ]:
# Criando a Confusion Matrix
cM <- confusionMatrix(teste$is_attributed, previsao)
cM
cM$byClass["Precision"]   # True Positive / (True Positive + False Positive)
cM$byClass["Recall"]      # True Positive / (True Positve + False Negative )
cM$byClass["F1"]          # 2 * (Precision * Recall) / (Precision + Recall)

---
#### > **Interpretando o resultado**
* O modelo previu corretamente 29558 vezes que o clique sem download
* O modelo previu corretamente 48 vezes o clique seguido de download  
  
  
* O modelo previu erradamente 16 vezes que um clique teria download
* O modelo previu erradamente 378 vezes que um clique não teria download  
  
---
  
    
#### > **Score**
* **Acurácia** - Número total de previsões corretas comparado com o total da amostra:         
    > 98.72%
* **Recall**   - Número de acertos da classe positiva (0): 
    > 99.93%
* **Precisão** - Total de previsões corretas para classe positiva (0):
    > 98.78%
* **Specificidade** - Total de previsões corretas para classe negativa (1):
    > 70.59%
* **Taxa de Falso Positivo** - Número de previsões incorretas para a classe positiva (1) dividido pelo total de membros da classe negativa (0):
    > 88.35%
* **F1 Score** - Média armônica entre a Precisão e a Acurácia:
    > 99.35%


In [ ]:
# Gerando a curva ROC

pred <- prediction(as.numeric(previsao), teste$is_attributed)
perf <- performance(pred, "tpr","fpr")

plot(perf, col = rainbow(10), main = "Curva ROC")
abline(a=0, b=1)

In [ ]:
# Gerando a Curva de Precision/Recall

perf <- performance(pred, "prec", "rec")
plot(perf, main = "Curva Precision/Recall")

---
#### > **Conclusão**
> O modelo apresentou alta pontuação ao prever cliques sem download (Classe 0): **98,78% de precisão**

> O mesmo não se pode dizer na performance de previsão para cliques com download (Classe 1): **70,59% de especificidade**

> Além disso, apresentou uma alta taxa de falso positivo: **88,53%**
   
> Tal comportamente é provavelmente fruto do desbalanceamento entre as classes: **99,77% dos dados de treinos são da Classe 0**

> 